# Building Machine Learning Classifiers: Evaluate Gradient Boosting with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [7]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [20, 30], 
    'max_depth': [3, 5],
    'learning_rate': [0.1]
}

clf = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = clf.fit(X_tfidf_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

c:\Users\hetia\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,19.644006,1.655701,0.162998,0.004924,0.1,5,30,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",0.954219,0.967684,0.959569,0.955975,0.955076,0.958505,0.004938,1
2,14.625981,0.317100,0.215400,0.039943,0.1,5,20,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",0.951526,0.955117,0.954178,0.948787,0.950584,0.952038,0.002324,2
1,15.580726,0.435284,0.218599,0.038758,0.1,3,30,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",0.950628,0.949731,0.949686,0.944295,0.937107,0.946289,0.005109,3
0,9.922599,0.316369,0.201729,0.022880,0.1,3,20,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",0.942549,0.942549,0.944295,0.936208,0.933513,0.939823,0.004189,4


In [5]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [20, 30], 
    'max_depth': [3, 5],
    'learning_rate': [0.1]
}

clf = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = clf.fit(X_count_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

c:\Users\hetia\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,19.681198,2.029936,0.160807,0.006764,0.1,5,30,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",0.955117,0.968582,0.958670,0.957772,0.955076,0.959043,0.004978,1
2,14.884138,1.141418,0.203399,0.018650,0.1,5,20,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",0.949731,0.956014,0.955076,0.949686,0.948787,0.951859,0.003043,2
1,15.615276,1.259295,0.243600,0.047869,0.1,3,30,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",0.954219,0.947935,0.947889,0.941599,0.939802,0.946289,0.005142,3
0,10.550599,0.524773,0.233001,0.059973,0.1,3,20,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",0.940754,0.942549,0.939802,0.934412,0.936208,0.938745,0.002995,4
